In [5]:
import sys
import pandas as pd
import tushare as ts
import sqlite3
import os
import glob
import copy
import datetime
import numpy as np
from struct import *

### 读取通达信安装文件下的沪深股票数据，存到sqlite3中，不推荐使用

In [4]:
def jiexiTDX_daily ( jiaoyisuo= 'SH',DB_Name = 'E:\sqlite3\\tdx.db'):
    '''
    读取通达信安装文件下的沪深股票数据，存到sqlite3中,数据库为tdx.db，table为sh_daily和sz_daily，不推荐使用
    '''
    if(jiaoyisuo=='SH'):    
        j_add='D:/Program Files (x86)/tdx/vipdoc/sh/lday/'
        sql='insert into sh_daily(ts_code,trade_date,open,close,high,low,vol,amount,refresh_date) values(?,?,?,?,?,?,?,?,?)'
        print("读取上交所股票信息。")
    elif(jiaoyisuo=='SZ'):
        j_add='D:/Program Files (x86)/tdx/vipdoc/sz/lday/'
        sql='insert into sz_daily(ts_code,trade_date,open,close,high,low,vol,amount,refresh_date) values(?,?,?,?,?,?,?,?,?)'
        print("读取深交所股票信息。")
    else:
        #j_add='D:/Program Files (x86)/tdx/vipdoc/ds/lday/'
        #sql="insert into index_daily (date_id,open1,high,low,close,amount,vol,code_id,refresh_date) values(%s,%s,%s,%s,%s,%s,%s,%s)"
        print("请使用其他函数。")   

    conn = sqlite3.connect(DB_Name)  
    cursor = conn.cursor()    # 向STUDENT表插入数据的SQL语句
    print('连接数据库%s成功' % (DB_Name))
    os.chdir(j_add)
    sh_list = glob.glob('*.day')
    #address = os.getcwd()#返回当前目录
    refresh_date = datetime.date.today() #今天日期

    for code in sh_list:
        dayfile = open(code, 'rb')
        buffer = dayfile.read()
        dayfile.close()
        code_id = code[2:-4]+"."+jiaoyisuo
        #读取的数据长度
        buflengh = len(buffer) #32的整数倍
        #除32获利文件中存储了多少天的信息
        datenum = int(buflengh / 32)
        #解析每一天的数据的内容,每一天列表的起始位置和终止位置
        list_begin = 0
        list_end = 32
        #创建pandas股票数据表
        #将个股数据转换为pandas的dataFrame格式数据
        dl =[]
        for K in range(datenum):
            #解析32个字节一天数据
            a = unpack('IIIIIfII', buffer[list_begin:list_end])
            date_id  = pd.to_datetime(str(a[0]),format='%Y-%m-%d').date()
            open1 = a[1] / 100.0
            high  = a[2] / 100.0
            low   = a[3] / 100.0
            close = a[4] / 100.0
            amount = round(a[5] / 100000.0, 2)
            vol = a[6]
            unused = a[7]
            #dl.append([code_id,date_id,close,open1,high,low,amount,vol,refresh_date])
            dl = [code_id,date_id,close,open1,high,low,amount,vol,refresh_date]
            #加32指向下一天的数据列表
            list_begin = list_begin + 32
            list_end = list_end + 32
            try:
                cursor.execute(sql,dl)    #每日的数据添加一次，不再汇总后添加
                #conn.commit()             
            except Exception as e:
                print(e)        
                conn.rollback() # 回滚
                print('插入数据到表失败')               
        print (code_id + ' ，has been saved in SQLite.TDX的 '+jiaoyisuo+'  交易所中!' )
    # 批量插入之后再执行事务提交
    conn.commit()   
    # 关闭游标
    cursor.close()
    # 断开数据库连接
    conn.close()

### 将dataframe数据存到sqlite3中

In [93]:
def Savetosqlite3(df,DB_Name = 'F:/sqlite3/tdx.db',sql='insert into test(ts_code,trade_date,close,open,high,low,pct_chg,vol,amount,name) values(?,?,?,?,?,?,?,?,?,?,?)'):    
    '''
    将dataframe数据存到sqlite3中
    '''
    
    conn = sqlite3.connect(DB_Name)  
    cursor = conn.cursor()    # 向sqlite3中表插入数据的SQL语句
    print('连接数据库%s成功' % (DB_Name))
    #print(df)
    refresh_date = datetime.date.today() #今天日期
    df['refresh_date'] = refresh_date
    
    try:
        df.drop(columns='结算价',inplace=True)
        print('已删除结算价格')
    except:
        print('无结算价')
    
    sqldata = [tuple(xi) for xi in df.values]
    
    try:
        cursor.executemany(sql,sqldata)    #每日的数据添加一次，不再汇总后添加
    except Unexpected as e:
        print(e)        
        conn.rollback() # 回滚
        print('插入数据到表失败')               
    print ( 'has been saved in SQLite.TDX的交易所中!' )
    # 批量插入之后再执行事务提交
    conn.commit()  
    # 关闭游标
    cursor.close()
    # 断开数据库连接
    conn.close()

### 读取通达信导出的TXT文件，存入sqlite3中，推荐使用

In [82]:
def save_TXTdata(file_address,DB_Name,sql):
    '''
    读取通达信导出的TXT文件，存入sqlite3中。通达信导出的数据中含有代码和名字，取国际指数和其他经济数据比较方便.tdx导出txt文本时，
    使用TAB（）格式，使用时间格式为YYYYMMDD,与tushare网站数据一致。建议使用TXT文本数据，方便数据库统一格式。
    '''
    os.chdir(file_address)
    files_list = glob.glob('*.txt')
    print(len(files_list))
    address = os.getcwd()#返回当前目录

    for files in files_list:
        try:
            #header =0,读入文件，txt文件中的第一行保存，然后代码和名称读出为nameN和nameM，skipfooter=1,删除最后一行的通达信数据源，delim_whitespace=True分隔符为tab
            data = pd.read_table(file_address+files,delim_whitespace=True,encoding='gb2312',skipfooter=1,engine='python',parse_dates=True,infer_datetime_format=True) 
            nameN = copy.deepcopy(data.columns[0])
            nameM = copy.deepcopy(data.columns[1])
            print(nameN,nameM)
            '''不要st股票,#name1 = re.sub(r'\/' , 'x',s1)
            if ('*st' in nameM):
                print(files_list[i] +' is *ST股票，删除中...')
                continue;
            '''
            #header =1，再次读入文件，包含txt文本中的第一行，并把刚刚读出的代码和名称写入
            data = pd.read_table(file_address+files,delim_whitespace=True,header =1,encoding='gb2312',skipfooter=1,engine='python',parse_dates=True,infer_datetime_format=True) 
            #data = pd.read_table(file_address +'\\'+ files_list[i],sep=',',encoding='gb2312',header =1,parse_dates=True,infer_datetime_format=True) 
            data.insert(0,'code',nameN)
            data.insert(8,'name',nameM)
            #data.rename(index=str, columns={"开盘": "open", "收盘": "close", "成交量": "vol", "日期": "date", "最低": "low", "最高": "high", "成交额": "amount"},inplace=True)
            #data.to_csv('D'+nameN+'.csv',sep=',',encoding='gb2312')
            data['pct_chg'] = round(data['收盘'].diff()/data['收盘'] * 100,4)
            data['成交量']   = round(data['成交量']/1000,4)
            #print(data)
            Savetosqlite3(data,DB_Name,sql)  #保存到数据库
            print( nameM  +' has been saved!' )
        except UnicodeDecodeError:
            print(files +'gb2312 codec can not decode')
        except:
            print(files +" has Unexpected error:")     

### 连接tushare.pro网站，爬取数据并存茹sqlite3

In [39]:
pro = ts.pro_api('31e6c92e6133d112d8ee557bb245044fde5daab04104598620f23921')
df = pro.index_daily(ts_code='399005.SZ', start_date='20200720', end_date='20200726')
#去掉代码中后三个字符，仅保留代码
df['ts_code'] = df['ts_code'].map(lambda ts_code: ts_code[:-3])

0    399005
1    399005
2    399005
3    399005
4    399005
Name: ts_code, dtype: object

### 读取sqlite3数据库数据

In [ ]:
con = sqlite3.connect("F:/sqlite3/tdx.db")
cur = con.cursor()

cur.execute("select * from index_daily where ts_code = '399300.SZ'")
#sqlite可识别的日期格式为“YYYY-MM-DD”
sql = "select * from test where trade_date Between '2020-07-01' AND '2020-07-24'"
df = cur.fetchmany(sql)
cur.close()
con.close()

In [95]:
if __name__ == "__main__":
    
    sql='insert into index_daily(ts_code,trade_date,open,high,low,close,vol,amount,name,pct_chg,refresh_date) values(?,?,?,?,?,?,?,?,?,?,?)'
    DB_Name = 'F:/sqlite3/tdx.db'
    file_address = 'F:/sqlite3/'
    
    save_TXTdata(file_address,DB_Name,sql)  
    #jiexiTDX_daily(jiaoyisuo='SZ',DB_Name = 'E:\sqlite3\\tdx.db')
    #jiexiTDX_daily(jiaoyisuo='SH',DB_Name = 'E:\sqlite3\\tdx.db')

24
A_IXIC 纳斯达克综合指数
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
纳斯达克综合指数 has been saved!
B_DAX 德国DAX指数
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
德国DAX指数 has been saved!
HSI 恒生指数
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
恒生指数 has been saved!
5_RMBUS 美元兑人民币
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
美元兑人民币 has been saved!
5_SHIBOR 上海同业拆借O/N
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
上海同业拆借O/N has been saved!
5_SHR1W 上海同业拆借1周
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
上海同业拆借1周 has been saved!
7_RQ 沪深融券余额(亿)
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
沪深融券余额(亿) has been saved!
7_RZ 沪深融资余额(亿)
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
沪深融资余额(亿) has been saved!
7_TON 累计北上资金(亿)
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been saved in SQLite.TDX的交易所中!
累计北上资金(亿) has been saved!
7_TOS 累计南下资金(亿)
连接数据库F:/sqlite3/tdx.db成功
无结算价
has been